In [2]:
from CF_SVD import SVD_CF
from CF_item_based import ItemBasedCF
from CF_user_based import UserBasedCF
from TwoTowerModel import TwoTowerModel
from dataset_processor import DatasetProcessor
from dataset_statistics import DatasetStatistics
import pandas as pd

D:\Anaconda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# movielens_data = pd.read_csv('processed_dataset/MovieLens-1M/ratings/fulldata_movielens.csv')
# train_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/traindata_movielens.csv')
# test_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/testdata_movielens.csv')

train_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ratings_traindata_movielens.csv')
test_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ratings_testdata_movielens.csv')

In [3]:
k = 5

user_cf = UserBasedCF(train_ratings)

Shape of Sparse User-Item matrix: (6040, 3667)
Shape of Sparse User Similarity matrix: (6040, 6040)


In [4]:
# recommendations_movielens = cf.generate_recommendations(test_ratings, k)

In [ ]:
def get_recommendations(self, user_id, k):
    if user_id not in self.user_similarity_df.index:
        return pd.Series()

    # Get the similarity scores for the user and filter out users with similarity <= 0
    similar_users = self.user_similarity_df[user_id][self.user_similarity_df[user_id] > 0].sort_values(
        ascending=False)

    if similar_users.empty:
        return pd.Series()  # Return an empty series if no similar users with similarity > 0

    # Get indices of similar users
    similar_users_indices = similar_users.index

    # Create a sparse matrix of user ratings filtered by similar users
    user_ratings_sparse = self.user_item_matrix.loc[similar_users_indices].to_numpy()
    user_similarities = similar_users.to_numpy()

    # Calculate weighted ratings
    weighted_ratings = user_ratings_sparse.T.dot(user_similarities) / user_similarities.sum()

    # Normalize weighted ratings to be between 1 and 5
    min_rating = 1
    max_rating = 5
    weighted_ratings = min_rating + (max_rating - min_rating) * (weighted_ratings - weighted_ratings.min()) / (
            weighted_ratings.max() - weighted_ratings.min())

    # Convert the weighted ratings back to a Series with item IDs as index
    weighted_ratings_series = pd.Series(weighted_ratings, index=self.user_item_matrix.columns)

    # Exclude items the user has already rated
    already_rated = self.user_item_matrix.loc[user_id][self.user_item_matrix.loc[user_id] > 0].index
    recommendations = weighted_ratings_series.drop(already_rated).sort_values(ascending=False).head(k)

    return recommendations

In [5]:
import numpy as np

def dcg_at_k(y_true_sorted, k):
    y_true_sorted = np.asarray(y_true_sorted)[:k]
    return np.sum((2**y_true_sorted - 1) / np.log2(np.arange(2, len(y_true_sorted) + 2)))

def ndcg_at_k(y_true, y_score, k):
    order = np.argsort(y_score)[::-1]
    y_true_sorted = np.take(y_true, order[:k])
    dcg = dcg_at_k(y_true_sorted, k)
    idcg = dcg_at_k(sorted(y_true, reverse=True), k)
    return dcg / idcg if idcg > 0 else 0.0

def recall_at_k(y_true, y_score, k):
    top_k_items = np.argsort(y_score)[::-1][:k]
    hits = np.sum(y_true[top_k_items])
    return hits / np.sum(y_true)

In [8]:
def evaluate_user_based_cf_model(model, test_data, k):
    ndcg_scores = []
    recall_scores = []

    # Get unique users
    unique_users = test_data['user_id'].unique()

    for user in unique_users:
        # Get items and labels for the user
        user_data = test_data[test_data['user_id'] == user]
        item_ids = user_data['item_id'].values
        labels = user_data['rating'].values

        # Get recommendations for the user
        recommendations = model.get_recommendations(user, k)

        if recommendations.empty:
            continue

        # Create a score array where recommended items have a high score
        scores = np.zeros_like(item_ids, dtype=float)
        for i, item_id in enumerate(item_ids):
            if item_id in recommendations.index:
                scores[i] = recommendations[item_id]

        # Calculate NDCG and Recall
        ndcg = ndcg_at_k(labels, scores, k)
        recall = recall_at_k(labels, scores, k)

        ndcg_scores.append(ndcg)
        recall_scores.append(recall)

    avg_ndcg = np.mean(ndcg_scores)
    avg_recall = np.mean(recall_scores)

    return {
        'NDCG@{}'.format(k): avg_ndcg,
        'Recall@{}'.format(k): avg_recall
    }

# Evaluate the model
eval_result = evaluate_user_based_cf_model(user_cf, test_ratings, k=5)
print(eval_result)

C:\Users\Hooman\AppData\Local\Temp\ipykernel_32936\4188945968.py:17: RuntimeWarning: invalid value encountered in scalar divide
  return hits / np.sum(y_true)
C:\Users\Hooman\AppData\Local\Temp\ipykernel_32936\4188945968.py:17: RuntimeWarning: invalid value encountered in scalar divide
  return hits / np.sum(y_true)
C:\Users\Hooman\AppData\Local\Temp\ipykernel_32936\4188945968.py:17: RuntimeWarning: invalid value encountered in scalar divide
  return hits / np.sum(y_true)
C:\Users\Hooman\AppData\Local\Temp\ipykernel_32936\4188945968.py:17: RuntimeWarning: invalid value encountered in scalar divide
  return hits / np.sum(y_true)
C:\Users\Hooman\AppData\Local\Temp\ipykernel_32936\4188945968.py:17: RuntimeWarning: invalid value encountered in scalar divide
  return hits / np.sum(y_true)
C:\Users\Hooman\AppData\Local\Temp\ipykernel_32936\4188945968.py:17: RuntimeWarning: invalid value encountered in scalar divide
  return hits / np.sum(y_true)
C:\Users\Hooman\AppData\Local\Temp\ipykernel_3

{'NDCG@5': 0.863445572289752, 'Recall@5': nan}


In [9]:
len(test_ratings)

102759

In [ ]:
from sklearn.metrics import ndcg_score
# import numpy as np

# Prepare test data for NDCG calculation
test_ratings['predicted_rating'] = test_ratings.apply(lambda row: recommendations_movielens.get(row['user_id'], pd.Series()).get(row['item_id'], 0), axis=1)
test_ratings_grouped = test_ratings.groupby('user_id')

# ndcg_scores = []
#
# for user, group in test_ratings_grouped:
#     true_ratings = group['rating'].values
#     pred_ratings = group['predicted_rating'].values
#     ndcg_scores.append(ndcg_score([true_ratings], [pred_ratings], k=10))
#
# user_cf_ndcg = np.mean(ndcg_scores)
# print("User-Based CF NDCG:", user_cf_ndcg)

In [ ]:
ndcg_scores = []

for user, group in test_ratings_grouped:
    if len(group) > 1:  # Filter users with more than one rating
        true_ratings = group['rating'].values
        pred_ratings = group['predicted_rating'].values
        ndcg_scores.append(ndcg_score([true_ratings], [pred_ratings], k=5))



In [ ]:
user_cf_ndcg = np.mean(ndcg_scores)
print("User-Based CF NDCG:", user_cf_ndcg)

baliee jadide

In [55]:
user_cf_ndcg = np.mean(ndcg_scores)
print("User-Based CF NDCG:", user_cf_ndcg)

User-Based CF NDCG: 0.8280448809270077


In [ ]:
# model.to('cuda')  # Ensure the model is on the correct device

# Initialize the metric evaluator
metric_evaluator = Metric(k=[5, 10, 20])

# Evaluate the model
eval_result = metric_evaluator.eval(model, test_dataloader, topks=[5, 10, 20])

# Print the results
for metric, values in eval_result.items():
    print(f"{metric}: {values}")

In [57]:
test_ratings

,user_id,item_id,rating,timestamp,predicted_rating
0,6040,2384,4,956703954,0.000000
1,6040,573,4,956704056,0.000000
2,6040,1734,2,956704081,0.000000
3,6040,912,5,956704191,0.000000
4,6040,2028,5,956704519,1.361253
...,...,...,...,...,...
200037,424,2581,3,1046349099,0.000000
200038,5950,3654,3,1046368480,0.000000
200039,5950,3328,3,1046369090,0.000000
200040,5950,3481,4,1046369090,0.000000


In [23]:
from evaluator import EvaluateMetrics

evaluator = EvaluateMetrics(movielens_testset)
results = evaluator.evaluate_recommendations(recommendations_movielens, k)
print(f"NDCG@{k}: {results['NDCG@k']:.4f}")
print(f"MRR@{k}: {results['MRR@k']:.4f}")
print(f"HR@{k}: {results['HR@k']:.4f}")

NDCG@5: 0.7092
MRR@5: 0.6328
HR@5: 1.0000


In [26]:
from evaluator import EvaluateMetrics

evaluator = EvaluateMetrics(movielens_testset)
results = evaluator.evaluate_recommendation_without_history(recommendations_movielens, k)
print(f"NDCG@{k}: {results['NDCG@k']:.4f}")
print(f"MRR@{k}: {results['MRR@k']:.4f}")
print(f"HR@{k}: {results['HR@k']:.4f}")

NDCG@5: 0.4126
MRR@5: 0.3682
HR@5: 0.5818


In [1]:
from CF_SVD import SVD_CF
from CF_item_based import ItemBasedCF
from CF_user_based import UserBasedCF
from TwoTowerModel import TwoTowerModel
from dataset_processor import DatasetProcessor
from dataset_statistics import DatasetStatistics
import pandas as pd

D:\Anaconda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
fields = ['rating', 'user_id', 'item_id', 'timestamp']

In [3]:
dat_file_path = 'datasets/ml-1m/ml-1m/ratings.dat'
processor_dat = DatasetProcessor(dat_file_path, fields, 'movielens')

In [4]:
processor_dat.load_data()
processor_dat.sort_by_column(column_name="timestamp")

Data sorted by 'timestamp' in ascending order.


In [5]:
stats = DatasetStatistics(processor_dat.data)

Info of the dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 1000209 entries, 1000138 to 825603
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1000209 non-null  int64
 1   item_id    1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 38.2 MB


In [6]:
stats.show_duplicates()
stats.drop_duplicates()

Duplicated ratings:
Empty DataFrame
Columns: [user_id, item_id, rating, timestamp]
Index: []


In [7]:
processor_dat.filter_data(min_ratings=5)

# unique USERS who have rated 5 or more products : 6040
# unique USERS dropped      : 0
# unique ITEMS remaining    : 3706
# unique ITEMS dropped      : 0

Final length of the dataset : 1000209


In [8]:
processor_dat.binarize_ratings(threshold=3)

Ratings binarized with threshold 3.


In [9]:
processor_dat.split_data()
processor_dat.save_data(output_dir='.', prefix='rating')

Full data set saved to: .\rating_fulldata_movielens.csv
Train set saved to: .\rating_traindata_movielens.csv
Validation set saved to: .\rating_valdata_movielens.csv
Test set saved to: .\rating_testdata_movielens.csv


In [15]:
train_df = pd.read_csv('rating_traindata_movielens.csv')
val_df = pd.read_csv('rating_valdata_movielens.csv')
test_df = pd.read_csv('rating_testdata_movielens.csv')

In [10]:
# Concatenate all datasets to calculate the total counts per user
all_data = pd.concat([train_df, val_df, test_df])

# Calculate the total count of records for each user in the entire dataset
total_counts = all_data.groupby('user_id').size().reset_index(name='total_count')

# Calculate the count of records for each user in the train set
train_counts = train_df.groupby('user_id').size().reset_index(name='train_count')

# Calculate the count of records for each user in the validation set
val_counts = val_df.groupby('user_id').size().reset_index(name='val_count')

# Calculate the count of records for each user in the test set
test_counts = test_df.groupby('user_id').size().reset_index(name='test_count')

# Merge all counts into a single DataFrame
merged_counts = total_counts.merge(train_counts, on='user_id', how='left').merge(val_counts, on='user_id', how='left').merge(test_counts, on='user_id', how='left')

# Fill NaN values with 0 (in case a user doesn't appear in a specific set)
merged_counts.fillna(0, inplace=True)

# Calculate the percentage for each set
merged_counts['train_percentage'] = merged_counts['train_count'] / merged_counts['total_count']
merged_counts['val_percentage'] = merged_counts['val_count'] / merged_counts['total_count']
merged_counts['test_percentage'] = merged_counts['test_count'] / merged_counts['total_count']

# Check the distribution
print(merged_counts)

# Optionally, you can save the results to a CSV file
merged_counts.to_csv('user_distribution_check.csv', index=False)

# Display any users who do not follow the 80-10-10 distribution (within a tolerance)
tolerance = 0.05
incorrect_distribution = merged_counts[
    (abs(merged_counts['train_percentage'] - 0.8) > tolerance) |
    (abs(merged_counts['val_percentage'] - 0.1) > tolerance) |
    (abs(merged_counts['test_percentage'] - 0.1) > tolerance)
]

# print("Users with incorrect distribution:")
# print(incorrect_distribution)

      user_id  total_count  train_count  val_count  test_count  \
0           1           53           42          5           6   
1           2          129          103         13          13   
2           3           51           40          5           6   
3           4           21           16          2           3   
4           5          198          158         20          20   
...       ...          ...          ...        ...         ...   
6035     6036          888          710         89          89   
6036     6037          202          161         20          21   
6037     6038           20           16          2           2   
6038     6039          123           98         12          13   
6039     6040          341          272         34          35   

      train_percentage  val_percentage  test_percentage  
0             0.792453        0.094340         0.113208  
1             0.798450        0.100775         0.100775  
2             0.784314        0.0

In [11]:
val_df

,user_id,item_id,rating,timestamp
0,1,745,3,978824268
1,1,595,5,978824268
2,1,588,4,978824268
3,1,1,5,978824268
4,1,2687,3,978824268
...,...,...,...,...
99687,6040,3083,4,963272132
99688,6040,2366,3,963272166
99689,6040,3819,5,963272166
99690,6040,1900,5,964828352


In [10]:
def dcg(scores, k):
    """
    Compute Discounted Cumulative Gain (DCG) for the top-k results.

    :param scores: List of relevance scores.
    :param k: Number of top results to consider.
    :return: DCG value.
    """
    scores = np.asfarray(scores)[:k]
    return np.sum(scores / np.log2(np.arange(2, scores.size + 2)))

def ndcg(scores, k):
    """
    Compute Normalized Discounted Cumulative Gain (NDCG) for the top-k results.

    :param scores: List of relevance scores.
    :param k: Number of top results to consider.
    :return: NDCG value.
    """
    ideal_scores = sorted(scores, reverse=True)
    return dcg(scores, k) / dcg(ideal_scores, k)

In [14]:
# Given relevance scores
relevance_scores = [1, 0, 0, 0, 0]
k = len(relevance_scores)
ndcg_value = ndcg(relevance_scores, k)

print(f"NDCG@{k}: {ndcg_value}")

NDCG@5: 1.0


In [5]:
import numpy as np
test_data = pd.DataFrame({
    'user': [1, 1, 1, 2, 2],
    'item': [101, 102, 103, 201, 202],
    'label': [1, 0, 1, 1, 0]
})
all_items = np.array([100, 101, 102, 103, 104, 200, 201, 202])
recommended_items = {
    1: [101, 102, 104, 103, 105],  # Items recommended for user 1
    2: [201, 202, 200, 203, 204]   # Items recommended for user 2
}


In [13]:
import pandas as pd
import numpy as np

# Example data
test_data = pd.DataFrame({
    'user': [1, 1, 1, 2, 2],
    'item': [101, 102, 103, 201, 202],
    'label': [3, 4, 5, 1, 2]
})

all_items = np.array([100, 101, 102, 103, 104, 200, 201, 202])

recommended_items = {
    1: [101, 102, 104, 103, 105],  # Items recommended for user 1
}

# User 1 example
user = 1
user_test_data = test_data[test_data['user'] == user]
test_items = user_test_data['item'].values
labels = user_test_data['label'].values

recommended_items_user = recommended_items[user]

y_score = [
    user_test_data[user_test_data['item'] == item]['label'].values[0] if item in test_items else 2.5
    for item in recommended_items_user
]

print("y_score for user 1:", y_score)

y_score for user 1: [3, 4, 2.5, 5, 2.5]


In [12]:
import torch

# Define the ndcg function
def ndcg(scores, labels, k):
    scores = scores.cpu()
    labels = labels.cpu()
    print(scores, labels)
    rank = (-scores).argsort(dim=1)
    print(rank)
    cut = rank[:, :k]
    hits = labels.gather(1, cut)
    print(hits)
    position = torch.arange(2, 2+k)
    print(position)
    weights = 1 / torch.log2(position.float())
    print(weights)
    dcg = (hits.float() * weights).sum(1)
    idcg = torch.Tensor([weights[:min(int(n), k)].sum() for n in labels.sum(1)])
    ndcg = dcg / idcg
    return ndcg.mean()

# Example usage:
# Predicted scores for 3 samples (each with 5 items)
scores = torch.tensor([[0.2, 0.5, 0.3, 0.7, 0.1]])

# Corresponding labels (1 indicates relevant, 0 indicates not relevant)
labels = torch.tensor([[0, 1, 0, 0, 0]])

# Calculate NDCG for top 3 items
k = 5
ndcg_value = ndcg(scores, labels, k)
print(f"NDCG@{k}: {ndcg_value.item()}")

tensor([[0.2000, 0.5000, 0.3000, 0.7000, 0.1000]]) tensor([[0, 1, 0, 0, 0]])
tensor([[3, 1, 2, 0, 4]])
tensor([[0, 1, 0, 0, 0]])
tensor([2, 3, 4, 5, 6])
tensor([1.0000, 0.6309, 0.5000, 0.4307, 0.3869])
NDCG@5: 0.6309297680854797
